# Electricity & Weather Data Analysis

# Introduction
This project is meant to gather insights on electricity usage.

This file will look for insights into electricity usage using weather data from the same time frame.

## Data
* This uses data cleaned with "green_button_data_cleaning.ipynb": clean_energy_use_*.csv
* This uses data cleaned with "weather_data_cleaning.ipynb.ipynb": clean_weather_*.csv


## Original Energy Data Source
Data is from my energy company(ComEd) from the past year. 10_22_2022 to 10_22_2023
Data from the [My Green Button](https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

## Original Weather Data Source
This data was collected using [Meteostat](https://github.com/meteostat/meteostat-python). The Meteostat Python library provides a simple API for accessing open weather and climate data. The historical observations and statistics are collected by Meteostat from different public interfaces, most of which are governmental.

Among the data sources are national weather services like the National Oceanic and Atmospheric Administration (NOAA) and Germany's national meteorological service (DWD).

# Column / header info


In [212]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hourly Usage Analysis

## Warm Months
It seems our electricity usage directly correlates with the sunlight hours of the day(8am-8pm). This makes sense for 2 reasons:
* We'd use the air conditioner for cooling during those hours.
* Those are the awake hours, so we'd use appliances during those hours.

## Cold Months
* Heating/Cooling USAGE should inversely correlate with sunlight hours.
* Other appliance USAGE still should correlate with sunlight hours.
These 2 USAGE timelines inversely correlate.


## Comparison
Cold months have about 3x the electricity USAGE than warm months. This likely has 2 causes.
* Chicago winters are much worse than the summers.
* Our heaters are MUCH less efficient than our air conditioners.